In [19]:
from citylearn.citylearn import CityLearnEnv
import torch
from a2c import A2C
from collections import deque

In [69]:
torch.set_num_threads(1)

dataset_name = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(dataset_name, central_agent=False, simulation_end_time_step=1000)

agents = [
        A2C(i, osp, asp, lr=3e-4, adam_eps=0.001, recurrent_policy=False, num_steps=1000, num_processes=1, device='cpu')
        for i, (osp, asp) in enumerate(zip(env.observation_space, env.action_space))
    ]

obs = torch.tensor(env.reset())
for i in range(len(obs)):
    agents[i].storage.obs[0].copy_(obs[i])
    agents[i].storage.to('cpu')

In [70]:
num_updates = (
    5
)

all_infos = deque(maxlen=10)

for j in range(1, num_updates + 1):

    for step in range(1000):
        # Sample actions
        with torch.no_grad():
            n_value, n_action, n_action_log_prob, n_recurrent_hidden_states = zip(
                *[
                    agent.model.act(
                        agent.storage.obs[step],
                        agent.storage.recurrent_hidden_states[step],
                        agent.storage.masks[step],
                    )
                    for agent in agents
                ]
            )

        print(n_action)
        # Obser reward and next obs
        obs, reward, done, infos = env.step(n_action)
        # envs.envs[0].render()

        # If done then clean the history of observations.
        masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])

        bad_masks = torch.FloatTensor(
            [
                [0.0] if info.get("TimeLimit.truncated", False) else [1.0]
                for info in infos
            ]
        )
        for i in range(len(agents)):
            agents[i].storage.insert(
                obs[i],
                n_recurrent_hidden_states[i],
                n_action[i],
                n_action_log_prob[i],
                n_value[i],
                reward[:, i].unsqueeze(1),
                masks,
                bad_masks,
            )

        for info in infos:
            if info:
                all_infos.append(info)

    # value_loss, action_loss, dist_entropy = agent.update(rollouts)
    for agent in agents:
        agent.compute_returns()

    for agent in agents:
        loss = agent.update([a.storage for a in agents])
        for k, v in loss.items():
            print(f"agent{agent.agent_id}/{k}", v, j)

    for agent in agents:
        agent.storage.after_update()

    # if save_interval is not None and (
    #     j > 0 and j % save_interval == 0 or j == num_updates
    # ):
    #     cur_save_dir = path.join(save_dir, f"u{j}")
    #     for agent in agents:
    #         save_at = path.join(cur_save_dir, f"agent{agent.agent_id}")
    #         os.makedirs(save_at, exist_ok=True)
    #         agent.save(save_at)
    #     archive_name = shutil.make_archive(cur_save_dir, "xztar", save_dir, f"u{j}")
    #     shutil.rmtree(cur_save_dir)

    # if eval_interval is not None and (
    #     j > 0 and j % eval_interval == 0 or j == num_updates
    # ):
    #     evaluate(
    #         agents, os.path.join(eval_dir, f"u{j}"),
    #     )

env.close()

(tensor([[0]]), tensor([[0]]), tensor([[0]]), tensor([[0]]), tensor([[0]]))


TypeError: '<=' not supported between instances of 'Tensor' and 'numpy.ndarray'